In [1]:
import torch
from torch import nn
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
import cProfile

In [2]:
x = torch.rand(1, 256, 16, 16, 16)
conv = nn.Conv3d(256, 128, 3)

In [6]:
inp, mat1, mat2 = torch.rand(64, 64), torch.rand(64, 64), torch.rand(64, 64)
addmm = torch.addmm
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = addmm(inp, mat1, mat2)
print(addmm)
print(prof.key_averages().table())


linear = nn.Linear(512, 256)
linear.weight = torch.nn.Parameter(torch.rand_like(linear.weight))
x = torch.randn(4, 8, 512)

with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = linear(x)
    y.mean().backward()
print(linear)
print(prof.key_averages().table())

conv = nn.Conv3d(256, 128, 3)
conv.weight = torch.nn.Parameter(torch.rand_like(conv.weight))
x = torch.rand(1, 256, 16, 16, 16)
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    y = conv(x)
    y.mean().backward()
print(conv)
print(prof.key_averages().table())

<built-in method addmm of type object at 0x7ff7db469a40>
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
           aten::addmm        83.13%      69.000us       100.00%      83.000us      83.000us             1  
           aten::copy_        16.87%      14.000us        16.87%      14.000us      14.000us             1  
    aten::resolve_conj         0.00%       0.000us         0.00%       0.000us       0.000us             3  
----------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 83.000us

Linear(in_features=512, out_features=256, bias=True)
-------------------------------------------------------  ------------  ------------  -----------

STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2023-11-01 05:58:30 41309:41309 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [4]:
print(prof.key_averages().table())

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.52%     431.000us        60.26%      50.273ms      50.273ms             1  
                                      aten::convolution         0.03%      27.000us        59.74%      49.842ms      49.842ms             1  
                                     aten::_convolution         0.04%      34.000us        59.71%      49.815ms      49.815ms             1  
                               aten::mkldnn_convolution        59.63%      49.753ms        59.67%      49.781ms      49.781ms             1  
      